In [9]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import sys 
import os
sys.path.append(os.path.abspath("/media/sf_Project2/Code"))
from IO import Input
from IO import Output
import pandas as pd

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [10]:
with cd("/media/sf_Project2/Code"): 
    train_dataset = np.array(Input.load_trainset_caffefeatures(featureSelectionMethod='RF',Percentile = 100)).astype('float32')
    train_labels = np.array(Input.load_trainset_labels()).astype('float32')
    valid_dataset = np.array(Input.load_validationset_caffefeatures(featureSelectionMethod='RF',Percentile = 100)).astype('float32')
    valid_labels = np.array(Input.load_validationset_labels()).astype('float32')

In [11]:
num_labels=10
train_labels = np.squeeze((np.arange(num_labels) == train_labels[:,None]).astype(np.float32))
valid_labels = np.squeeze((np.arange(num_labels) == valid_labels[:,None]).astype(np.float32))
train_labels0 = train_labels[:,1]
train_labels0 = train_labels0.reshape((train_labels.shape[0],1))

print(train_labels0)
#print(train_dataset)
#train_labels = train_labels.reshape((train_labels.shape[0],1))
#valid_labels = valid_labels.reshape((valid_labels.shape[0],1))
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
feature_size = train_dataset.shape[1]
print(train_labels)

[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]
Training set (14363, 3983) (14363, 10)
Validation set (8061, 3983) (8061, 10)
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


In [24]:
import scipy as sp
def accuracy(pred,labels):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(labels*sp.log(pred) + sp.subtract(1,labels)*sp.log(sp.subtract(1,pred)))
    ll = np.sum(ll * -1.0/len(labels))
    
    return ll 

# Network:

In [25]:
batch_size = 64
hlSize0 = 516
beta = 0.008
#decay_steps = 200
#decay_rate = 0.90
#learningStart=0.0007
decay_steps = 150
decay_rate = 0.97
learningStart=0.00012

stdv = 0.03
#patch_size = 5
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
  global_step = tf.Variable(0)  # count the number of steps taken.

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize0],
            stddev=stdv))
  input_biases = tf.Variable(tf.zeros([hlSize0]))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [hlSize0,num_labels], 
            stddev=stdv))
  layer1_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))

  # Model.
  def model(data):
    layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    layer2 = tf.matmul(layer1, layer1_weights) + layer1_biases
    return layer2
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta * tf.nn.l2_loss(input_weights) + \
    beta * tf.nn.l2_loss(layer1_weights)
  #  beta * tf.nn.l2_loss(layer2_weights) + \
  #  beta * tf.nn.l2_loss(layer3_weights) + \
  #  beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  learning_rate = tf.train.exponential_decay(learningStart, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [26]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")
  input_weights_val = input_weights.eval()
  input_biases_val = input_biases.eval()
  layer1_weights_val = layer1_weights.eval()
  layer1_biases_val = layer1_biases.eval()
  valid_prediction_val = valid_prediction.eval()

Initialized
Minibatch loss at step 0: 8.107173
Minibatch accuracy: 3.4%
Validation accuracy: 3.7%
Minibatch loss at step 100: 8.065252
Minibatch accuracy: 3.3%
Validation accuracy: 3.4%
Minibatch loss at step 200: 8.209554
Minibatch accuracy: 3.5%
Validation accuracy: 3.3%
Minibatch loss at step 300: 7.653537
Minibatch accuracy: 2.8%
Validation accuracy: 3.3%
Minibatch loss at step 400: 8.063700
Minibatch accuracy: 3.3%
Validation accuracy: 3.3%
Minibatch loss at step 500: 8.642140
Minibatch accuracy: 3.9%
Validation accuracy: 3.2%
Minibatch loss at step 600: 7.619357
Minibatch accuracy: 2.8%
Validation accuracy: 3.2%
Minibatch loss at step 700: 7.457805
Minibatch accuracy: 2.6%
Validation accuracy: 3.2%
Minibatch loss at step 800: 7.962366
Minibatch accuracy: 3.2%
Validation accuracy: 3.1%
Minibatch loss at step 900: 7.198700
Minibatch accuracy: 2.3%
Validation accuracy: 3.1%
Minibatch loss at step 1000: 8.528196
Minibatch accuracy: 3.8%
Validation accuracy: 3.1%
Minibatch loss at ste

In [27]:
validData = pd.DataFrame(valid_prediction_val)
Output.to_outputfile(validData,1,'NNlogloss14valid')

In [17]:
with cd("/media/sf_Project2/Code"): 
    test_data = np.array(Input.load_testdata_caffefeatures(True,range(30000),'RF',100)).astype('float32')

In [18]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    firstHalfTest = tf.nn.softmax(layer2).eval()
print(firstHalfTest)

[[ 0.0596142   0.14439088  0.3612155  ...,  0.08696077  0.03702684
   0.03995219]
 [ 0.24477209  0.1297767   0.02517665 ...,  0.04445281  0.05572335
   0.14426808]
 [ 0.40642512  0.07195009  0.13021336 ...,  0.03209694  0.22179855
   0.10249869]
 ..., 
 [ 0.10858535  0.18343051  0.16120921 ...,  0.07069349  0.14549027
   0.11613334]
 [ 0.09417057  0.02365284  0.08147081 ...,  0.0475096   0.08872037
   0.01363371]
 [ 0.02660265  0.09868814  0.25429115 ...,  0.02856197  0.01648503
   0.00358483]]


In [19]:
with cd("/media/sf_Project2/Code"): 
    test_data = np.array(Input.load_testdata_caffefeatures(True,range(30000,60000),'RF',100)).astype('float32')

In [20]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    secondHalfTest = tf.nn.softmax(layer2).eval()
print(secondHalfTest)

[[ 0.01888872  0.03283195  0.12382396 ...,  0.13200016  0.03280394
   0.11089096]
 [ 0.23953389  0.0255143   0.06045777 ...,  0.01913744  0.06200473
   0.05665646]
 [ 0.0293878   0.32002831  0.20523925 ...,  0.11942246  0.09858142
   0.03220553]
 ..., 
 [ 0.31247187  0.03845941  0.01839172 ...,  0.038016    0.05029628
   0.12308078]
 [ 0.28168941  0.0094756   0.03100446 ...,  0.03016209  0.05470277
   0.22167306]
 [ 0.26376244  0.08530376  0.05559533 ...,  0.01588675  0.01901895
   0.2209778 ]]


In [21]:
with cd("/media/sf_Project2/Code"): 
    test_data = np.array(Input.load_testdata_caffefeatures(True,range(60000,80000),'RF',100)).astype('float32')

In [22]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    thirdHalfTest = tf.nn.softmax(layer2).eval()
print(thirdHalfTest)

[[ 0.17438526  0.06614044  0.07201292 ...,  0.04925891  0.07350317
   0.07848214]
 [ 0.22405468  0.03417123  0.15462524 ...,  0.1291174   0.07858931
   0.08211536]
 [ 0.22022519  0.09074793  0.03218605 ...,  0.03081081  0.03026744
   0.04086332]
 ..., 
 [ 0.03600797  0.04970362  0.03733748 ...,  0.01686293  0.03669351
   0.13310859]
 [ 0.04982862  0.09211095  0.21691097 ...,  0.26551589  0.07713404
   0.03341889]
 [ 0.08659285  0.04745078  0.01191301 ...,  0.00914811  0.0173691
   0.02606795]]


In [23]:
testClass = np.concatenate([firstHalfTest,secondHalfTest,thirdHalfTest],0)
testClass = pd.DataFrame(testClass)
Output.to_outputfile(testClass,1,"NNSTRUCTUR11FULLtestset",validation=False)